<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.1


    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66


    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-09-19 10:18:44
-------------------
qualified stocks: 86
with latest results: 26
still star stocks: 15
-------------------
Initial Investment:  1.22 C
CY Investment:  1.49 C
Reserve:  6.17 L
Current:  1.43 C
-------------------
Today PnL: -14.55 K (-0.1%)
Current PnL: -16.90 L (-11.37%)
CY Booked + Current PnL: -5.46 L (-3.67%)
-------------------
Total profit:  2.03 L
Total loss:  -18.93 L
-------------------
Total Booked + Current PnL: 21.52 L (17.69%)
Total Booked PnL: 38.42 L (31.57%)
Curr Year Booked PnL: 11.44 L (8.02%)
Prev Year Booked PnL: 26.98 L (22.17%)
Est FTT:  2.24 C
Est FTT PnL: 81.41 L (57.05%)
Deployed:  1.22 C
Current:  1.43 C
CAGR/XIRR %: 9.85%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
14,BLUESTARCO,2080.00,12.69,61.0,H-MC,3.47,194570.0,29900.0,13425.0,-0.78,18.16,6.90,26.31,89.0,2.23,1.43,27.59,X40N,NTT,AC
76,TTKPRESTIG,770.00,102.99,53.0,M-SC,3.70,87436.0,-13341.0,13430.0,-0.91,-13.24,15.36,0.09,245.0,-0.99,0.64,14.87,OX40N,NTT,DURABLES
83,VOLTAS,1530.00,1.58,59.0,H-MC,2.68,213045.0,21303.0,16447.0,-0.41,11.11,7.72,19.69,99.0,1.30,1.56,18.66,XY25,NTT,AC
49,NATIONALUM,244.55,-43.66,65.0,H-MC,7.94,111920.0,11416.0,17695.0,-0.06,11.36,15.81,28.96,79.0,0.65,0.82,49.18,MH,ATH,METALS
56,RELIANCE,1533.00,-12.54,55.0,H-LC,4.95,219102.0,8736.0,20048.0,-0.74,4.15,9.15,13.68,37.0,0.44,1.60,21.36,XY25,NTT,REFINERIES


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAMMAANCAP,326.00,-175.00,70.0,M-SC,3.32,86040.0,-16170.0,109563.0,3.02,-15.82,127.34,91.37,208.0,-0.15,0.63,39.67,XY25,NTT,FINANCE
78,UNIONBANK,163.00,-13.18,65.0,M-LC,7.04,157702.0,16862.0,27629.0,1.54,11.97,17.52,31.59,66.0,0.61,1.16,41.63,XY24,NTT,BANKS
26,GREENPANEL,537.00,233.33,61.0,M-SC,1.81,148962.0,-34116.0,113092.0,1.53,-18.63,75.92,43.14,230.0,-0.30,1.09,40.12,XY24,NTT,MISC
59,SAIL,228.00,47.55,69.0,M-MC,7.52,232711.0,7749.0,159896.0,1.47,3.44,68.71,74.53,192.0,0.05,1.70,36.81,XY24,BTT,STEEL
17,CAMS,5211.76,1.26,69.0,H-SC,3.45,115192.0,13188.0,30733.0,0.95,12.93,26.68,43.06,122.0,0.43,0.84,32.26,X40N,ATH,MISC


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,RAJESHEXPO,518.00,1850.76,60.0,L-SC,2.68,53223.0,-83954.0,84050.0,-2.01,-61.20,157.92,0.07,267.0,-1.00,0.39,32.37,OX40N,NTT,JEWELLERY
81,VALIANTORG,838.00,-324.58,53.0,H-SC,6.26,124814.0,-38791.0,160111.0,-1.99,-23.71,128.28,74.15,139.0,-0.24,0.91,59.99,XR,NTT,CHEMICALS
36,INDIAMART,4850.92,-52.96,35.0,H-SC,11.49,131175.0,7839.0,125928.0,-1.90,6.36,96.00,108.45,119.0,0.06,0.96,30.13,AR,ATH,MISC
52,QUESS,986.00,-40.50,47.0,M-SC,39.83,58867.0,-6139.0,157069.0,-1.39,-9.44,266.82,232.18,198.0,-0.04,0.43,2.52,XY24,NTT,MISC
80,VAIBHAVGBL,521.00,63.64,67.0,H-SC,6.62,141682.0,-41093.0,152166.0,-1.37,-22.48,107.40,60.77,125.0,-0.27,1.04,26.90,XR,NTT,JEWELLERY


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
35,IEX,219.00,-29.46,56.0,H-SC,16.92,201623.0,1669.0,95569.0,-0.02,0.83,47.40,48.63,115.0,0.02,1.48,12.48,XR,NTT,MISC
34,ICICIPRULI,790.00,-20.31,48.0,H-MC,2.17,137046.0,1258.0,41498.0,0.23,0.93,30.28,31.48,107.0,0.03,1.00,13.16,X40,ATH,INSURANCE
29,HAVELLS,2069.16,0.91,61.0,H-MC,1.94,250177.0,2446.0,72601.0,0.59,0.99,29.02,30.30,92.0,0.03,1.83,14.41,X40,ATH,ELECTRICAL
51,PGHH,17973.08,-31.34,48.0,H-MC,4.11,198840.0,-1980.0,70747.0,-0.73,-0.99,35.58,34.25,80.0,-0.03,1.46,4.16,X40,ATH,FMCG


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
76,TTKPRESTIG,770.0,102.99,53.0,M-SC,3.70,87436.0,-13341.0,13430.0,-0.91,-13.24,15.36,0.09,245.0,-0.99,0.64,14.87,OX40N,NTT,DURABLES
77,UJJIVANSFB,60.0,94.87,54.0,H-SC,9.41,123120.0,-19359.0,38881.0,0.20,-13.59,31.58,13.70,163.0,-0.50,0.90,45.32,OX40N,NTT,BANKS
44,KANSAINER,340.0,-66.69,67.0,H-SC,1.24,229545.0,-40122.0,76461.0,0.71,-14.88,33.31,13.47,138.0,-0.52,1.68,16.10,XY24,NTT,PAINTS
18,CERA,9475.0,-20.05,46.0,H-SC,2.10,145659.0,-30244.0,72261.0,-0.23,-17.19,49.61,23.89,149.0,-0.42,1.07,25.80,OX40N,NTT,CERAMICS
71,TATAELXSI,9161.0,-19.85,53.0,H-MC,7.45,108386.0,-23449.0,65671.0,-0.51,-17.79,60.59,32.03,98.0,-0.36,0.79,21.40,OX40N,NTT,IT


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
59,SAIL,228.00,47.55,69.0,M-MC,7.52,232711.0,7749.0,159896.0,1.47,3.44,68.71,74.53,192.0,0.05,1.70,36.81,XY24,BTT,STEEL
14,BLUESTARCO,2080.00,12.69,61.0,H-MC,3.47,194570.0,29900.0,13425.0,-0.78,18.16,6.90,26.31,89.0,2.23,1.43,27.59,X40N,NTT,AC
83,VOLTAS,1530.00,1.58,59.0,H-MC,2.68,213045.0,21303.0,16447.0,-0.41,11.11,7.72,19.69,99.0,1.30,1.56,18.66,XY25,NTT,AC
17,CAMS,5211.76,1.26,69.0,H-SC,3.45,115192.0,13188.0,30733.0,0.95,12.93,26.68,43.06,122.0,0.43,0.84,32.26,X40N,ATH,MISC
29,HAVELLS,2069.16,0.91,61.0,H-MC,1.94,250177.0,2446.0,72601.0,0.59,0.99,29.02,30.30,92.0,0.03,1.83,14.41,X40,ATH,ELECTRICAL


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
49,NATIONALUM,244.55,-43.66,65.0,H-MC,7.94,111920.0,11416.0,17695.0,-0.06,11.36,15.81,28.96,79.0,0.65,0.82,49.18,MH,ATH,METALS
84,WHIRLPOOL,2270.00,-40.79,50.0,M-SC,0.65,100042.0,8544.0,70209.0,-0.11,9.34,70.18,86.07,223.0,0.12,0.73,45.28,XR,NTT,DURABLES
85,WIPRO,420.00,-9.71,58.0,M-LC,5.66,158726.0,7781.0,101680.0,-0.36,5.15,64.06,72.51,53.0,0.08,1.16,11.49,XR,NTT,IT
36,INDIAMART,4850.92,-52.96,35.0,H-SC,11.49,131175.0,7839.0,125928.0,-1.90,6.36,96.00,108.45,119.0,0.06,0.96,30.13,AR,ATH,MISC
11,BANKINDIA,190.00,-27.95,66.0,H-MC,6.32,184745.0,4937.0,107485.0,0.93,2.75,58.18,62.52,88.0,0.05,1.35,37.77,XR,NTT,BANKS


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
82,VBL,671.64,-16.43,42.0,H-LC,6.58,301232.0,-14610.0,127271.0,-0.30,-4.63,42.25,35.67,5.0,-0.11,2.21,8.59,X40N,ATH,FMCG
5,ASIANPAINT,3465.66,-13.06,47.0,H-LC,7.78,219578.0,-32190.0,85394.0,0.67,-12.79,38.89,21.13,27.0,-0.38,1.61,18.11,X40,ATH,PAINTS
41,ITC,452.00,-37.92,50.0,H-LC,2.38,200710.0,572.0,20312.0,-0.27,0.29,10.12,10.44,4.0,0.03,1.47,5.87,X40,NTT,FMCG
19,CIPLA,1795.00,-19.78,50.0,H-LC,5.70,213610.0,9110.0,32298.0,-1.20,4.45,15.12,20.25,10.0,0.28,1.56,13.58,X40N,BTT,PHARMA
50,NESTLEIND,1377.00,-9.11,52.0,H-LC,4.18,279513.0,14087.0,42710.0,-1.21,5.31,15.28,21.40,11.0,0.33,2.05,13.03,XY25,NTT,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
47,LTIM,7230.2,0.29,70.0,H-LC,1.48,248782.0,2631.0,76575.0,0.07,1.07,30.78,32.18,16.0,0.03,1.82,37.50,X200,ATH,IT
41,ITC,452.0,-37.92,50.0,H-LC,2.38,200710.0,572.0,20312.0,-0.27,0.29,10.12,10.44,4.0,0.03,1.47,5.87,X40,NTT,FMCG
50,NESTLEIND,1377.0,-9.11,52.0,H-LC,4.18,279513.0,14087.0,42710.0,-1.21,5.31,15.28,21.40,11.0,0.33,2.05,13.03,XY25,NTT,FMCG
56,RELIANCE,1533.0,-12.54,55.0,H-LC,4.95,219102.0,8736.0,20048.0,-0.74,4.15,9.15,13.68,37.0,0.44,1.60,21.36,XY25,NTT,REFINERIES
19,CIPLA,1795.0,-19.78,50.0,H-LC,5.70,213610.0,9110.0,32298.0,-1.20,4.45,15.12,20.25,10.0,0.28,1.56,13.58,X40N,BTT,PHARMA


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
79,UNITDSPR,1644.00,-12.66,53.0,H-LC,7.75,229104.0,-5040.0,53656.0,0.35,-2.15,23.42,20.77,86.0,-0.09,1.68,4.62,X40N,NTT,BREWERIES
73,TCS,4476.75,-25.47,58.0,H-LC,12.29,297557.0,-48399.0,123248.0,-0.35,-13.99,41.42,21.64,1.0,-0.39,2.18,5.41,X40,ATH,IT
41,ITC,452.00,-37.92,50.0,H-LC,2.38,200710.0,572.0,20312.0,-0.27,0.29,10.12,10.44,4.0,0.03,1.47,5.87,X40,NTT,FMCG
82,VBL,671.64,-16.43,42.0,H-LC,6.58,301232.0,-14610.0,127271.0,-0.30,-4.63,42.25,35.67,5.0,-0.11,2.21,8.59,X40N,ATH,FMCG
30,HCLTECH,1943.91,1.94,52.0,H-LC,9.70,232862.0,-9062.0,72327.0,-0.71,-3.75,31.06,26.15,8.0,-0.13,1.71,10.04,X40,ATH,IT


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SIEMENS,7969.85,3.60,65.0,H-LC,26.99,166325.0,-19770.0,232173.0,0.26,-10.62,139.59,114.13,15.0,-0.09,1.22,22.48,AR,ATH,ELECTRICAL
41,ITC,452.00,-37.92,50.0,H-LC,2.38,200710.0,572.0,20312.0,-0.27,0.29,10.12,10.44,4.0,0.03,1.47,5.87,X40,NTT,FMCG
19,CIPLA,1795.00,-19.78,50.0,H-LC,5.70,213610.0,9110.0,32298.0,-1.20,4.45,15.12,20.25,10.0,0.28,1.56,13.58,X40N,BTT,PHARMA
56,RELIANCE,1533.00,-12.54,55.0,H-LC,4.95,219102.0,8736.0,20048.0,-0.74,4.15,9.15,13.68,37.0,0.44,1.60,21.36,XY25,NTT,REFINERIES
5,ASIANPAINT,3465.66,-13.06,47.0,H-LC,7.78,219578.0,-32190.0,85394.0,0.67,-12.79,38.89,21.13,27.0,-0.38,1.61,18.11,X40,ATH,PAINTS


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4476.75,-25.47,58.0,H-LC,12.29,297557.0,-48399.0,123248.0,-0.35,-13.99,41.42,21.64,1.0,-0.39,2.18,5.41,X40,ATH,IT
39,INFY,2275.00,-16.37,55.0,H-LC,8.18,325272.0,12384.0,159318.0,-0.88,3.96,48.98,54.87,3.0,0.08,2.38,10.87,X40,BTT,IT
41,ITC,452.00,-37.92,50.0,H-LC,2.38,200710.0,572.0,20312.0,-0.27,0.29,10.12,10.44,4.0,0.03,1.47,5.87,X40,NTT,FMCG
82,VBL,671.64,-16.43,42.0,H-LC,6.58,301232.0,-14610.0,127271.0,-0.30,-4.63,42.25,35.67,5.0,-0.11,2.21,8.59,X40N,ATH,FMCG
1,ABB,7934.00,-37.53,68.0,H-LC,9.85,261058.0,-561.0,119773.0,0.05,-0.21,45.88,45.57,7.0,-0.00,1.91,11.08,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.00,7022.22,65.0,L-SC,5.20,78148.0,-15662.0,92277.0,0.75,-16.70,118.08,81.67,269.0,-0.17,0.57,51.34,XR,NTT,CERAMICS
21,COFFEEDAY,80.00,-43.66,51.0,L-SC,42.23,84576.0,-28973.0,69022.0,0.20,-25.52,81.61,35.27,268.0,-0.42,0.62,107.00,XR,NTT,HOTELS
60,SAMMAANCAP,326.00,-175.00,70.0,M-SC,3.32,86040.0,-16170.0,109563.0,3.02,-15.82,127.34,91.37,208.0,-0.15,0.63,39.67,XY25,NTT,FINANCE
3,ALKYLAMINE,4546.37,-3.83,49.0,H-SC,12.50,89492.0,-11471.0,106003.0,0.28,-11.36,118.45,93.63,148.0,-0.11,0.66,33.30,SR,ATH,CHEMICALS
48,MASFIN,398.61,-17.34,50.0,H-SC,12.98,94080.0,-3900.0,25505.0,-0.16,-3.98,27.11,22.05,152.0,-0.15,0.69,36.75,XR,ATH,FINANCE


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
63,SHALBY,327.0,1096.17,68.0,M-SC,4.53,162599.0,-18544.0,64015.0,-0.29,-10.24,39.37,25.10,235.0,-0.29,1.19,30.31,XY24,NTT,HEALTHCARE
60,SAMMAANCAP,326.0,-175.00,70.0,M-SC,3.32,86040.0,-16170.0,109563.0,3.02,-15.82,127.34,91.37,208.0,-0.15,0.63,39.67,XY25,NTT,FINANCE
7,ATULAUTO,844.0,3862.16,62.0,M-SC,5.35,123522.0,-23846.0,75657.0,-0.90,-16.18,61.25,35.16,236.0,-0.32,0.90,26.17,XY24,NTT,AUTO
74,TITAGARH,1548.0,0.96,68.0,H-SC,3.16,165480.0,-26535.0,105427.0,-0.10,-13.82,63.71,41.08,158.0,-0.25,1.21,40.22,XY24,NTT,ENGINEERING
80,VAIBHAVGBL,521.0,63.64,67.0,H-SC,6.62,141682.0,-41093.0,152166.0,-1.37,-22.48,107.40,60.77,125.0,-0.27,1.04,26.90,XR,NTT,JEWELLERY


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.10
1,25,43.93
2,50,74.51


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.53
LC    32.12
MC    23.34
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     29.38
X40      15.41
X40N     11.96
XR       11.78
AR        8.74
XY25      8.25
OX40N     8.20
X200      1.82
MH        1.70
X5K       1.45
SR        1.30
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    29.28
H-LC    25.40
H-MC    20.33
M-SC    13.67
M-LC     5.66
M-MC     2.70
L-SC     1.58
L-LC     1.06
L-MC     0.31
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
IT,13.64,-12.81,74.32
FMCG,11.90,-4.30,40.80
FINANCE,9.76,-15.42,60.63
MISC,7.47,-12.60,75.19
BANKS,6.35,-15.07,75.61
PAINTS,5.90,-14.76,31.86
ELECTRICAL,5.79,-4.14,61.53
AC,3.71,4.35,11.66
INSURANCE,3.45,-6.70,43.29


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2832893.0,22
AR,1302783.0,9
XR,1300288.0,14
X40,744892.0,10
X40N,573782.0,9
OX40N,527210.0,10
XY25,337568.0,6
SR,244233.0,2
X5K,119725.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3412042.0,29
M-SC,1378732.0,17
H-LC,1224783.0,14
H-MC,1093841.0,15
M-LC,378241.0,4
M-MC,308957.0,2
L-SC,245349.0,3
L-MC,60498.0,1
L-LC,38988.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1022326.0      6
M-SC       XY24       796596.0      7
H-SC       AR         789594.0      5
           XR         754406.0      7
H-MC       XY24       488498.0      4
H-LC       X40        460599.0      5
           AR         351946.0      2
M-MC       XY24       308957.0      2
H-SC       X40N       285648.0      4
           OX40N      252048.0      4
           SR         244233.0      2
H-MC       X40        213324.0      4
H-LC       X40N       213225.0      3
L-SC       XR         161299.0      2
M-SC       AR         161243.0      2
M-LC       XY24       156836.0      2
H-MC       XY25       126259.0      2
M-SC       OX40N      125441.0      4
M-LC       X5K        119725.0      1
M-SC       XR         114920.0      2
           XY25       109563.0      1
H-MC       XR         107485.0      1
M-LC       XR         101680.0      1
L-SC       OX40N       84050.0      1
H-LC       X200        76575.0      1
H-MC       X40N        74909.0      2
M-SC       X40         70969.0      1
H-MC       OX40N       65671.0      1
H-SC       MH          63787.0      1
H-LC       XY25        62758.0      2
L-MC       XR          60498.0      1
H-LC       XY24        59680.0      1
L-LC       XY25        38988.0      1
H-MC       MH          17695.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 25.0 seconds
